This is an attempt to partially replicate the model described in ["The Evolution of Layered Protocol Stacks Leads to an Hourglass-Shaped Architecture" (Saamer Akshabi, Constantine Dovrolis)](https://www.cc.gatech.edu/~dovrolis/Papers/evoarch.pdf), for fun.

In [ ]:
from typing import List

def default_generalities(layers: int = 10) -> List[float]:
    return [(layers - i - 1) / layers for i in range(layers)]

print(f'Default generality vector: {default_generalities()}')

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Params:
    generalities: List[float] = field(default_factory=default_generalities)  # aka s(l)
    competition_threshold: float = 0.6  # aka c
    init_nodes_per_layer: int = 10
    birth_rate: float = 0.05
    mortality: float = 1.0  # aka z
    min_rounds: int = 100
    max_nodes: int = 500

print(f'Default model params: {Params()}')

In [6]:
from __future__ import annotations
from typing import Dict, Set

@dataclass(eq=False)
class Node:
    layer: Layer
    products: Set[Node] = field(default_factory=set)
    value: int = 0
    
    def update_value(self) -> None:
        if self.layer.is_top():
            self.value = 1
        else:
            self.value = sum(node.value for node in self.products)

@dataclass(eq=False)
class Layer:
    arch: Arch
    seq: int
    nodes: Set[Node]
    
    def create_node(self) -> Node:
        return self.nodes.add(Node(layer=self))
    
    def is_top():
        return self.seq == len(self.arch.layers) - 1

@dataclass(eq=False)
class Arch:
    layers: List[Layer] = field(default_factory=list)
    rounds: int = 0
    
    def create_layer(self) -> Layer:
        layer = Layer(arch=self, seq=len(self.layers))
        self.layers.append(layer)
        return layer

def update_value(arch: Arch, nid: int):
    node = arch.nodes[nid]

def create_arch(params):
    arch = Arch()
    nlayers = len(params.generalities)
    for i in range(nlayers):
        layer = arch.create_layer()
        for j in range(params.init_nodes_per_layer):
            layer.create_node()
    for layer in reversed(arch.layers):
        for node in layer.nodes:
            node.update_value()
    return arch